# Data Exploration

## Background 

### Data Source
The data used in this notebook is sourced from the National Centers for Environmental Information (NCEI): [Global Historical Climatology Network (GHCN) - Hourly](https://www.ncei.noaa.gov/products/global-historical-climatology-network-hourly). Refer to their documentation and terms of use.


#### Data Set

Station_ID: the station identification code. The first two characters signify the FIPS country code, the third character is a network code identifying the station numbering system used, and the remaining eight characters contain the actual station ID.

Station_Name: the name of the station.

Year: the year the observation was taken in Coordinated Universal Time (UTC).

Month: the month the observation was taken in Coordinated Universal Time (UTC).

Day: the day the observation was taken in Coordinated Universal Time (UTC).

Hour: the hour the observation was taken in Coordinated Universal Time (UTC).

Latitude: latitude of the station (in decimal degrees). North (+); South (-).

Longitude: the longitude of the station (in decimal degrees). East (+); West (-).

Temperature: 2 meter (circa) Above Ground Level Air (dry bulb) Temperature (⁰C to tenths)


Notes: 
- Raw data was removed in download_ghcn.py for storage purposes.
- GHCN hourly dataset contained psv files for individual stations in specific years. When processing the data, it was converted to csv format files for all California stations in years 2003 - 2023.
- Most columns were dropped as they were not needed. Columns kept were described above.
- Duplicate rows that had completely same column values were dropped.

# Data Cleaning

### Set up

In [1]:
import pandas as pd # type: ignore
import numpy as np
import sys
import os

# Update paths to get source code from notebook_utils
curr_dir = os.path.dirname(os.path.abspath('notebooks'))
proj_dir = os.path.dirname(curr_dir)
src_path = os.path.join(proj_dir, 'src')
sys.path.append(src_path)

from notebook_utils.preprocessing import *

CA_stations = get_full_df('../data/processed/ghcn_full', chunksize=500000)

Processed file: full_CA_stations_2003.csv
Processed file: full_CA_stations_2004.csv
Processed file: full_CA_stations_2005.csv
Processed file: full_CA_stations_2006.csv
Processed file: full_CA_stations_2007.csv
Processed file: full_CA_stations_2008.csv
Processed file: full_CA_stations_2009.csv
Processed file: full_CA_stations_2010.csv
Processed file: full_CA_stations_2011.csv
Processed file: full_CA_stations_2012.csv
Processed file: full_CA_stations_2013.csv
Processed file: full_CA_stations_2014.csv
Processed file: full_CA_stations_2015.csv
Processed file: full_CA_stations_2016.csv
Processed file: full_CA_stations_2017.csv
Processed file: full_CA_stations_2018.csv
Processed file: full_CA_stations_2019.csv
Processed file: full_CA_stations_2020.csv
Processed file: full_CA_stations_2021.csv
Processed file: full_CA_stations_2022.csv
Processed file: full_CA_stations_2023.csv


### Data Examination

In [2]:
CA_stations.head()

,Station_ID,Station_name,Year,Month,Day,Hour,Latitude,Longitude,temperature
0,GPW00000401,POINTE A PITRE INTL AP,2003,1,1,0,16.2669,-61.6,24.7
1,GPW00000401,POINTE A PITRE INTL AP,2003,1,1,1,16.2669,-61.6,25.0
2,GPW00000401,POINTE A PITRE INTL AP,2003,1,1,3,16.2669,-61.6,24.2
3,GPW00000401,POINTE A PITRE INTL AP,2003,1,1,4,16.2669,-61.6,24.0
4,GPW00000401,POINTE A PITRE INTL AP,2003,1,1,5,16.2669,-61.6,22.0


In [3]:
CA_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58090845 entries, 0 to 58090844
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Station_ID    object 
 1   Station_name  object 
 2   Year          int64  
 3   Month         int64  
 4   Day           int64  
 5   Hour          int64  
 6   Latitude      float64
 7   Longitude     float64
 8   temperature   float64
dtypes: float64(3), int64(4), object(2)
memory usage: 3.9+ GB


In [4]:
CA_stations.describe()

,Year,Month,Day,Hour,Latitude,Longitude,temperature
count,5.809084e+07,5.809084e+07,5.809084e+07,5.809084e+07,5.809084e+07,5.809084e+07,3.939865e+07
mean,2.013291e+03,6.504075e+00,1.573479e+01,1.155804e+01,3.617811e+01,-1.177964e+02,1.642611e+01
std,5.475781e+00,3.448708e+00,8.791612e+00,6.933797e+00,4.853230e+00,1.586318e+01,8.636838e+00
min,2.003000e+03,1.000000e+00,1.000000e+00,0.000000e+00,1.115000e+01,-1.242390e+02,-9.900000e+01
25%,2.009000e+03,4.000000e+00,8.000000e+00,6.000000e+00,3.440800e+01,-1.219586e+02,1.100000e+01
50%,2.014000e+03,7.000000e+00,1.600000e+01,1.200000e+01,3.723810e+01,-1.205438e+02,1.560000e+01
75%,2.018000e+03,1.000000e+01,2.300000e+01,1.800000e+01,3.885470e+01,-1.181500e+02,2.170000e+01
max,2.023000e+03,1.200000e+01,3.100000e+01,2.300000e+01,4.198800e+01,1.457700e+02,9.990000e+02


The temperature column has a really high max celsius value which is 902 degrees celsius. This is unreasonably high. After doing some searching, we found that the highest recorded temperature value was 56.7 degrees celsius in California 1913. 

There is also an unreasonably low temperature observation of -99 degrees celsius since the lowest recorded temperature observation on Earth was -98 degrees in Antartica. 

In [5]:
optimize_col_types(CA_stations)
CA_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58090845 entries, 0 to 58090844
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Station_ID    object 
 1   Station_name  object 
 2   Year          int16  
 3   Month         int8   
 4   Day           int8   
 5   Hour          int8   
 6   Latitude      float32
 7   Longitude     float32
 8   temperature   float32
dtypes: float32(3), int16(1), int8(3), object(2)
memory usage: 1.8+ GB


# Cleaning Invalid Data

- Handle Missing Values  (e.g., mean/median impuation, interpolation, forward or backward fill, k-nearest neighbors imputation, deletion)
- Handle Outliers  (e.g., visual inspection by boxplots, Z-score and IQR method, or data transformation by log transformation and winsorization)
- Handle inconsistencies (e.g., checking ranges to ensure temperature values fall within a reasonable range, unit consistency, string matching and standardization), and duplicates (identify and remove duplicates) in the dataset

Notes:
- For non-leap years, there should be 8760 rows (for each hour) for each station.
- For leap years, there should be 8784 rows (for each hour) for each station
- Leap years from 2003-2023 include: 2004, 2008, 2012, 2016, and 2020
- The reduced files contain 99 CA stations.
- Some stations are not observed each year from 2003-2023.


## Handling Inconsistencies

1. Ensure temperature observations are within -50°C and 60°C
2. Temperature values above the reasonable range will be converted to NaN.

In [6]:
# Check how many rows are below -50 and above 60 degrees celsius
CA_stations[(CA_stations['temperature'] < -50) | (CA_stations['temperature'] > 60)]

,Station_ID,Station_name,Year,Month,Day,Hour,Latitude,Longitude,temperature
4664225,USW00003154,CAMP PENDLETON MCAS,2005,10,10,0,33.304199,-117.355003,-89.0
4665844,USW00003154,CAMP PENDLETON MCAS,2005,11,14,22,33.304199,-117.355003,-78.0
4666040,USW00003154,CAMP PENDLETON MCAS,2005,11,19,3,33.304199,-117.355003,89.0
4803671,USW00023110,LEMOORE REEVES NAS,2005,2,7,3,36.333302,-119.949997,94.0
4803740,USW00023110,LEMOORE REEVES NAS,2005,2,9,5,36.333302,-119.949997,83.0
...,...,...,...,...,...,...,...,...,...
57892973,USW00023289,PALO ALTO,2023,2,8,17,37.466702,-122.116699,80.0
58011331,USW00093193,FRESNO YOSEMITE INTL,2023,5,15,16,36.779999,-119.720299,227.0
58020400,USW00093201,TRUCKEE AP,2023,5,23,15,39.320000,-120.139397,70.0
58067942,USW00093231,SAN CARLOS AP,2023,2,10,15,37.516701,-122.250000,70.0


In [7]:
# Write code to turn temperature outside of the specified range into NaN
CA_stations.loc[(CA_stations['temperature'] < -50) | (CA_stations['temperature'] > 60), 'temperature'] = np.nan

## Deleting Duplicate Rows

There are rows with the same value in each column except temperature. In these cases we will average out the temperature observations and delete the extra rows.

## Handling Missing Values

In [8]:
grouped_df = CA_stations.groupby(['Station_ID', 'Station_name', 'Year', 'Month', 'Day', 'Hour', 'Latitude', 'Longitude']).agg({'temperature': 'mean'}).reset_index()
CA_stations = grouped_df

### Filling missing rows

In [9]:
# create a reference dataframe with all stations with hours from 2003 to 2023
full_df = create_full_df()

391


In [10]:
full_df.describe()

,datetime,Year,Month,Day,Hour,Latitude,Longitude,temperature
count,69967104,6.996710e+07,6.996710e+07,6.996710e+07,6.996710e+07,0.0,0.0,0.0
mean,2013-03-16 23:30:00.000071680,2.012712e+03,6.451851e+00,1.572747e+01,1.150000e+01,NaN,NaN,NaN
min,2003-01-01 00:00:00,2.003000e+03,1.000000e+00,1.000000e+00,0.000000e+00,NaN,NaN,NaN
25%,2008-02-07 23:45:00,2.008000e+03,3.000000e+00,8.000000e+00,5.750000e+00,NaN,NaN,NaN
50%,2013-03-16 23:30:00,2.013000e+03,6.000000e+00,1.600000e+01,1.150000e+01,NaN,NaN,NaN
75%,2018-04-23 23:15:00,2.018000e+03,9.000000e+00,2.300000e+01,1.725000e+01,NaN,NaN,NaN
max,2023-05-31 23:00:00,2.023000e+03,1.200000e+01,3.100000e+01,2.300000e+01,NaN,NaN,NaN
std,NaN,5.896072e+00,3.455160e+00,8.799113e+00,6.922187e+00,NaN,NaN,NaN


In [11]:
# Fill in completely missing rows
missing_rows = find_missing_rows(full_df, CA_stations)
CA_stations = add_missing_rows(CA_stations, missing_rows)

In [12]:
error = check_station_rows(CA_stations)
print(error)

       Station_ID  Year  Row_Count  Expected_Row_Count
20    BBW00000408  2023       3628                3924
41    CQL000LLBP7  2023       3624                3924
62    DOW00000402  2023       3624                3924
83    DOW00000403  2023       3624                3924
104   GPW00000401  2023       3624                3924
...           ...   ...        ...                 ...
8126  USW00093242  2023       3624                3924
8147  USW00093243  2023       3624                3924
8168  USW00093244  2023       3624                3924
8189  USW00093245  2023       3624                3924
8210  USW00094299  2023       3624                3924

[391 rows x 4 columns]


In [31]:
CA_stations.tail()

,Station_ID,Station_name,Year,Month,Day,Hour,Latitude,Longitude,temperature
45142748,USW00094299,ALTURAS MUNI AP,2023,5,31,19,41.483601,-120.561401,18.9
45142749,USW00094299,ALTURAS MUNI AP,2023,5,31,20,41.483601,-120.561401,20.0
45142750,USW00094299,ALTURAS MUNI AP,2023,5,31,21,41.483601,-120.561401,21.1
69967135,USW00094299,ALTURAS MUNI AP,2023,5,31,22,41.483601,-120.561401,NaN
69967136,USW00094299,ALTURAS MUNI AP,2023,5,31,23,41.483601,-120.561401,NaN


### Station Deletion

## Filling missing temperature values

Stations with no Station_name values will be dropped as that means there are no observations recorded for them at all from 2003 - 2023
- This is because the above Station_name for missing rows were filled in by using the Station_name used from a filled column with the same Station_ID.

In [14]:
CA_stations.dropna(subset=['Station_name'], inplace=True) #353 stations left

In [15]:
null_count = CA_stations.isna().sum()
null_percent = (null_count/CA_stations.shape[0]) * 100
print(f'Percentage of null values for each column: \n{null_percent}')

Percentage of null values for each column: 
Station_ID       0.00000
Station_name     0.00000
Year             0.00000
Month            0.00000
Day              0.00000
Hour             0.00000
Latitude         0.00000
Longitude        0.00000
temperature     57.91542
dtype: float64


### Missing Temperature Values: Handling large gaps in temperature observations

We define a "large gap" as being a gap in the temperature column that is more than a day.

Large gaps in temperature observations will be handled via interpolation

#### Cubic Spline Interpolation for large gaps of missing temperature values

In [39]:
def cubic_spline_interpolate(data, gap_hours=24):

    data = data.copy()
    data['temperature'] = pd.to_numeric(data['temperature'], errors='coerce')

    # Convert to Datetime and set as index
    data['Datetime'] = pd.to_datetime(data[['Year', 'Month', 'Day', 'Hour']])
    data.set_index('Datetime', inplace=True)
    data.sort_index(inplace=True)

    # Resample to hourly data
    hourly_data = data['temperature'].resample('h').mean()
    is_missing = hourly_data.isna()

    missing_indices = np.where(is_missing)[0]
    gaps = np.split(missing_indices, np.where(np.diff(missing_indices) != 1)[0] + 1)

    interpolated_data = hourly_data.copy()

    for gap in gaps:
        if len(gap) >= gap_hours:
            start_index = max(gap[0] - 1, 0)
            end_index = min(gap[-1] + 1, len(hourly_data) - 1)

            x = np.arange(start_index, end_index + 1)
            y = hourly_data.iloc[start_index:end_index + 1].values
            mask = ~np.isnan(y)

            if np.sum(mask) > 1:
                # Apply cubic spline interpolation
                cs = CubicSpline(x[mask], y[mask])
                interpolated_data.iloc[gap] = cs(gap)

    data['temperature'] = data['temperature'].combine_first(interpolated_data)
    data['temperature'] = data['temperature'].round(1)
    data.reset_index(drop=True, inplace=True)  # Ensure the index is not dropped
    data.sort_values(by=['Station_ID', 'Year', 'Month', 'Day', 'Hour'], inplace=True)

    return data

CA_interpolated_df = cubic_spline_interpolate(CA_stations,gap_hours=24)

ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
CA_interpolated_df.tail()

,Station_ID,Station_name,Year,Month,Day,Hour,Latitude,Longitude,temperature
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
null_count = CA_interpolated_df.isna().sum()
null_percent = (null_count/CA_interpolated_df.shape[0]) * 100
print(f'Percentage of null values for each column: \n{null_percent}')

Percentage of null values for each column: 
Station_ID      100.0
Station_name    100.0
Year            100.0
Month           100.0
Day             100.0
Hour            100.0
Latitude        100.0
Longitude       100.0
temperature     100.0
dtype: float64


### Missing Temperature Values: Handling short gaps in temperature observations

We define a "short gap" as being a gap in the temperature column that is less than a day. 

Short gaps in temperature observations will be handled with forward/backward fill.

In [ ]:
final_df = fill_gaps(CA_interpolated_df)

In [ ]:
null_count = final_df.isna().sum()
null_percent = (null_count/final_df.shape[0]) * 100
print(f'Percentage of null values for each column: \n{null_percent}')

#### Validation

In [ ]:
import matplotlib.pyplot as plt

# make copies to not alter original dataframes
plot_CA_stations = CA_stations.copy()
plot_CA_interpolated = CA_interpolated_df.copy()

plot_CA_stations['Datetime'] = pd.to_datetime(plot_CA_stations[['Year', 'Month', 'Day', 'Hour']])
plot_CA_interpolated['Datetime'] = pd.to_datetime(plot_CA_interpolated[['Year', 'Month', 'Day', 'Hour']])

plot_CA_stations.set_index('Datetime', inplace=True)
plot_CA_interpolated.set_index('Datetime', inplace=True)

monthly_avg_original = plot_CA_stations['temperature'].resample('ME').mean()
monthly_avg_interpolated = plot_CA_interpolated['temperature'].resample('ME').mean()

plt.figure(figsize=(14, 7))

plt.plot(monthly_avg_original.index, monthly_avg_original.values, 'o-', label='Original Data', markersize=4)

plt.plot(monthly_avg_interpolated.index, monthly_avg_interpolated.values, 'x-', label='Interpolated Data', markersize=4)

plt.xlabel('Datetime')
plt.ylabel('Temperature (°C)')
plt.title('Monthly Average Temperature (2003-2023)')
plt.legend()
plt.grid(True)
plt.show()

## Handling Outliers

Outliers can skew our statistical analysis of the data. 

### 1. Visual Inspection

#### Boxplot

In [ ]:
import seaborn as sns

plt.figure(figsize=(12, 8))
sns.boxplot(x='Year', y='temperature', data=CA_interpolated_df)
plt.title('Temperature Boxplot by Year')
plt.xticks(rotation=90)  
plt.ylabel('Temperature')
plt.show()

### 2. Statistical Methods

#### Z-Score

In [ ]:
from scipy import stats

# z_stations dataframe will have a Z_score column added to it
z_stations = CA_interpolated_df.copy()
z_stations['Z_score'] = stats.zscore(z_stations['temperature'])
z_thresh = 3

# calculate outliers using Z-score
z_outliers = z_stations[(z_stations['Z_score'] < -z_thresh) | (z_stations['Z_score'] > z_thresh)]
total_observations = z_stations.shape[0]

num_outliers = z_outliers.shape[0]

percent_outliers = (num_outliers / total_observations) * 100

print(f'{percent_outliers:.2f}% of the observations are outliers')

### 3. Drop Outliers

In [ ]:
z_stations_cleaned = z_stations[~((z_stations['Z_score'] < -z_thresh) | (z_stations['Z_score'] > z_thresh))]

# Drop the Z_score column as it's no longer needed
z_stations_cleaned = z_stations_cleaned.drop(columns=['Z_score'])

# The z_stations_cleaned dataframe now contains the data without the outliers
z_stations_cleaned.describe()

### UTC to Local Time Conversion

In [ ]:
z_stations_cleaned['Datetime'] = pd.to_datetime(z_stations_cleaned[['Year', 'Month', 'Day', 'Hour']])
z_stations_cleaned = z_stations_cleaned.drop(columns=['Year', 'Month', 'Day', 'Hour'])

z_stations_cleaned['Datetime_local'] = z_stations_cleaned['Datetime'].dt.tz_localize('UTC').dt.tz_convert('US/Pacific')
z_stations_cleaned['Year'] = z_stations_cleaned['Datetime_local'].dt.year
z_stations_cleaned['Month'] = z_stations_cleaned['Datetime_local'].dt.month
z_stations_cleaned['Day'] = z_stations_cleaned['Datetime_local'].dt.day
z_stations_cleaned['Hour'] = z_stations_cleaned['Datetime_local'].dt.hour
z_stations_cleaned = z_stations_cleaned.drop(columns=['Datetime', 'Datetime_local'])

# remove 2002 observations
z_stations_cleaned = z_stations_cleaned[z_stations_cleaned['Year'] != 2002]

# rearrange columns
cols = ['Station_ID','Station_name', 'Latitude', 'Longitude','Year','Month','Day','Hour','temperature']
z_stations_cleaned = z_stations_cleaned[cols]

In [ ]:
z_stations_cleaned.head()   

### Write dataframe to csv files by year

In [ ]:
output_folder = '../data/processed/ghcn_clean'
os.makedirs(output_folder, exist_ok=True)

years = z_stations_cleaned['Year'].unique()

# Capitalize the temperature column to match the rest.
z_stations_cleaned = z_stations_cleaned.rename(columns={'temperature': 'Temperature'})
z_stations_cleaned.sort_values(by=['Station_ID', 'Year', 'Month', 'Day', 'Hour'], inplace=True)

for year in years:
    yearly_data = z_stations_cleaned[z_stations_cleaned['Year'] == year]
    output_file = os.path.join(output_folder, f'CA_{year}_clean.csv')
    yearly_data.to_csv(output_file, index=False)

print('New datafrane yearly files saved to ghcn_cleaned')